### Karpathy Let's Build a GPT Tokenizer 

This is me roughly following along with [Karpathy's BPE Video](https://www.youtube.com/watch?v=zduSFxRajkE&t=1430s). A cool OpenAI app for visualizing tokenization is [tiktokenizer](https://tiktokenizer.vercel.app/).

### Unicode/UTF-8

At a high level, strings in python are immutable sequences of unicode code points (use ord(char) to find). Encodings can take unicode and convert to bytestrings- UTF8, UTF16, UTF32. 

Here's more detail, coming from [Nathan Reed's blog](https://www.reedbeta.com/blog/programmers-intro-to-unicode/). Unicode has over 150k characters, 
as opposed to 256 ASCII. Unicode charactrers are "code points", whcih are often written /in hexadecimal with a prefix U+ -> U+0041 or something. The codespace consists of over 1 million code points, with only around 12% actually assigned. 

Encodings map hexdecimal index in codespace to bytes. The easiest way is to just store it as a 32-bit integer, which consumes 4 bytes per code point. This is lot of extra storage; UTF-32 = Unicode Transformation Format is a bit better but still a lot. More common are UTF-8/UTF-16, which are "variable length encodings"- 8 bit/16 bit units. Here, code points with smaller index values take up fewer bytes, but it is a bit slower because more involved. 

UTF-8: code points are stored in 1-4 bytes, based on index value. For example, the 128 ASCII characters are encoded as single bytes, so ASCII files can be interpreted as UTF-8 without conversion. 

Unicode also enables things like composing marks (combining two characters into one, canonical equivalence- equating equivalent characters, normalization forms- equivalence class of characters).

In [5]:
list(":) to test utf8".encode("utf-8"))

[58, 41, 32, 116, 111, 32, 116, 101, 115, 116, 32, 117, 116, 102, 56]

In [6]:
list("utf16".encode("utf-16"))

[255, 254, 117, 0, 116, 0, 102, 0, 49, 0, 54, 0]

Note lots of "wasteful" encoding to 0s. We generally want to use UTF-8 and then maybe further combine, i.e. enable us to change |V| as necessary

### Byte Pair Encoding

* input = aaabdaaabac
* vocab = {a, b, c, d}
* vocab_size = |vocab|
* while vocab_size > ideal_size:
    * Let z := most frequent pair (z = aa)
    * replace(z, aa) (seq = zabdzabac, vocab = {z,a,b,c,d}
* return vocab

In [11]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode('utf-8') 
print(f'Raw Bytes: {tokens[:5]}')
tokens = list(map(int, tokens))
print(f'Text: {text}') 
print(f'Len(text): {len(text)}')
print(f'=' *20)
print(f'Tokens: {tokens}')
print(f'Len(tokens): {len(tokens)}')
### each char is 1-4 bytes so 533 text -> 616 tokens makes sense

Raw Bytes: b'\xef\xbc\xb5\xef\xbd'
Text: Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
Len(text): 533
Tokens: [239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226,

In [18]:
from collections import Counter 

def find_most_common_pair(tokens):
    return Counter(zip(tokens, tokens[1:])).most_common(1)[0][0]

top_pair = find_most_common_pair(tokens)
print(f'Most Common Pair: {top_pair}, Chars: {chr(top_pair[0]), chr(top_pair[1])}')

Most Common Pair: (101, 32), Chars: ('e', ' ')


In [21]:
def merge(ids, pair, idx):
    ### list of ints (ids), replace(pair, idx)
    newids = []
    i = 0 
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2 
        else:
            newids.append(ids[i])
            i+=1 
    return newids

print(merge([5,4,5,1,2,5,4], (5,4), 104))


[104, 5, 1, 2, 104]


In [22]:
ideal_size = 275
num_merges = ideal_size - 256
ids = list(tokens)

merges = {}
for i in range(num_merges):
    pair = find_most_common_pair(ids)
    idx = 256 + i 
    print(f'merging {pair} into token {idx}')
    ids = merge(ids, pair, idx)
    merges[pair] = idx

merging (101, 32) into token 256
merging (240, 159) into token 257
merging (226, 128) into token 258
merging (105, 110) into token 259
merging (115, 32) into token 260
merging (97, 110) into token 261
merging (116, 104) into token 262
merging (257, 133) into token 263
merging (257, 135) into token 264
merging (97, 114) into token 265
merging (239, 189) into token 266
merging (258, 140) into token 267
merging (267, 264) into token 268
merging (101, 114) into token 269
merging (111, 114) into token 270
merging (116, 32) into token 271
merging (259, 103) into token 272
merging (115, 116) into token 273
merging (261, 100) into token 274
